In [3]:
# !pip install earthengine-api geemap

import ee
import geemap

# 1. Authenticate and Initialize
try:
    ee.Initialize(project='earth-engine-project-458018')
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project='earth-engine-project-458018')


# 2. Define Region of Interest (ROI)
roi = ee.Geometry.Rectangle([77.5, 12.8, 78.0, 13.1])  #  Bangalore area


# 3. Cloud Masking Function for Sentinel-2 SR
def maskS2sr(image):
    # Use Scene Classification Layer (SCL) to remove clouds, shadows, snow
    scl = image.select('SCL')
    mask = scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10))  # Remove cloud shadow, medium cloud, cirrus, etc.
    return image.updateMask(mask)


# 4. Load Raw & Processed Sentinel-2 Images
def load_images(start_date, end_date):
    raw = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 40))

    processed = raw.map(maskS2sr)
    return raw.median().clip(roi), processed.median().clip(roi)

# Load for 2018 and 2024
raw_2018, image_2018 = load_images('2018-01-01', '2018-12-31')
raw_2024, image_2024 = load_images('2024-01-01', '2024-12-31')


# 5. Compute NDBI and Urban Masks
ndbi_2018 = image_2018.normalizedDifference(['B11', 'B8']).rename('NDBI_2018')
ndbi_2024 = image_2024.normalizedDifference(['B11', 'B8']).rename('NDBI_2024')

urban_2018 = ndbi_2018.gt(0.2).selfMask()
urban_2024 = ndbi_2024.gt(0.2).selfMask()


# 6. Urban Expansion Detection
expansion = urban_2024.And(urban_2018.Not())


# 7. Visualize Everything
Map = geemap.Map(center=[12.95, 77.7], zoom=10)

# True color visual parameters
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}

# Add layers: raw and cloud-masked
Map.addLayer(raw_2018, vis_params, "2018 Raw (Before Cloud Removal)")
Map.addLayer(image_2018, vis_params, "2018 Processed (After Cloud Removal)")

Map.addLayer(raw_2024, vis_params, "2024 Raw (Before Cloud Removal)")
Map.addLayer(image_2024, vis_params, "2024 Processed (After Cloud Removal)")

# Urban area layers
Map.addLayer(urban_2018, {'palette': ['yellow']}, "Urban Areas 2018")
Map.addLayer(urban_2024, {'palette': ['blue']}, "Urban Areas 2024")
Map.addLayer(expansion, {'palette': ['red']}, "Urban Expansion (New Urban Areas)")

# Region of interest
Map.addLayer(roi, {}, 'ROI')

# Show map
Map


Map(center=[12.95, 77.7], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…